In [5]:
import random, itertools, math

# Parameters
n = 8  # Octagon
vertices = list(range(n))

def is_adjacent(a, b):
    return (a + 1) % n == b or (b + 1) % n == a

def crosses(a, b, c, d):
    # Convex polygon crossing test
    if len({a, b, c, d}) < 4:
        return False
    if is_adjacent(a, b) or is_adjacent(c, d):
        return False

    def between(x, y, z):
        return (x < z and x < y < z) or (z < x and (y > x or y < z))

    return between(a, c, b) != between(a, d, b) and between(c, a, d) != between(c, b, d)

def canonical(a, b):
    return (a, b) if a < b else (b, a)

# All possible diagonals
all_diagonals = [canonical(a, b) for a in vertices for b in vertices
                 if b > a and not is_adjacent(a, b)]

# Generate all non‑crossing sets of 5 diagonals (triangulations)
triangulations = []
for combo in itertools.combinations(all_diagonals, 5):
    if all(not crosses(*d1, *d2) for i, d1 in enumerate(combo)
           for d2 in combo[i + 1:]):
        triangulations.append(combo)

# Pre‑compute, for each triangulation, which triangles use which diagonals
edge_set_cache = {}
triangles_cache = {}

def triangles_in_triangulation(diags):
    if diags in triangles_cache:
        return triangles_cache[diags]
    edges = {canonical(i, (i + 1) % n) for i in vertices}
    edges.update(diags)
    tris = []
    for a in vertices:
        for b in vertices:
            if b <= a or canonical(a, b) not in edges:
                continue
            for c in vertices:
                if c <= b:
                    continue
                if canonical(a, c) in edges and canonical(b, c) in edges:
                    # Identify the diagonals bounding this triangle
                    diag_edges = [e for e in (canonical(a, b), canonical(b, c), canonical(a, c)) if e in diags]
                    tris.append(tuple(diag_edges))
    triangles_cache[diags] = tris
    return tris

# Simulation
def random_game():
    # Pick a random triangulation & random order of its 5 diagonals
    tr = random.choice(triangulations)
    order = list(tr)
    random.shuffle(order)          # Draw order of diagonals
    when = {d: t for t, d in enumerate(order)}
    p1 = p2 = 0
    for tri_diags in triangles_in_triangulation(tr):
        last = max(when[d] for d in tri_diags)
        if last % 2 == 0:      # turns 0,2,4 → Player 1
            p1 += 1
        else:
            p2 += 1
    return p1, p2

NUM_TRIALS = 100_000
p1_wins = p2_wins = ties = 0
for _ in range(NUM_TRIALS):
    s1, s2 = random_game()
    if s1 > s2:
        p1_wins += 1
    elif s2 > s1:
        p2_wins += 1
    else:
        ties += 1

print(f"After {NUM_TRIALS:,} random games:")
print(f"  Player 1 wins ≈ {p1_wins / NUM_TRIALS:.4f}")
print(f"  Player 2 wins ≈ {p2_wins / NUM_TRIALS:.4f}")
print(f"  Ties           ≈ {ties      / NUM_TRIALS:.4f}")


After 100,000 random games:
  Player 1 wins ≈ 0.6082
  Player 2 wins ≈ 0.0248
  Ties           ≈ 0.3670


In [8]:
import itertools, math, pandas as pd

n = 8
vertices = list(range(n))

def is_adjacent(a, b):
    return (a + 1) % n == b or (b + 1) % n == a

coords = [(math.cos(2*math.pi*i/n), math.sin(2*math.pi*i/n)) for i in vertices]

def orient(i, j, k):
    x1, y1 = coords[i]
    x2, y2 = coords[j]
    x3, y3 = coords[k]
    return (x2 - x1)*(y3 - y1) - (y2 - y1)*(x3 - x1)

def crosses(a, b, c, d):
    if len({a, b, c, d}) < 4:
        return False
    o1 = orient(a, b, c)
    o2 = orient(a, b, d)
    o3 = orient(c, d, a)
    o4 = orient(c, d, b)
    return o1 * o2 < 0 and o3 * o4 < 0

def canon(a, b):
    return (a, b) if a < b else (b, a)

all_diagonals = [canon(i, j) for i in vertices for j in vertices
                 if i < j and not is_adjacent(i, j)]

triangulations = []
for combo in itertools.combinations(all_diagonals, 5):
    if all(not crosses(*d1, *d2) for d1, d2 in itertools.combinations(combo, 2)):
        triangulations.append(combo)

polygon_edges = {canon(i, (i+1)%n) for i in vertices}

def triangles_of(tri_diags):
    edges = set(polygon_edges).union(tri_diags)
    tris = []
    for a, b, c in itertools.combinations(vertices, 3):
        e1, e2, e3 = canon(a, b), canon(a, c), canon(b, c)
        if e1 in edges and e2 in edges and e3 in edges:
            diag_edges = tuple(sorted([e for e in (e1, e2, e3) if e in tri_diags]))
            tris.append(diag_edges)
    return tris

p1_wins = p2_wins = ties = 0

for tri in triangulations:
    triangles = triangles_of(tri)
    for order in itertools.permutations(tri):
        turn_index = {d: t for t, d in enumerate(order)}
        p1 = p2 = 0
        for diag_tuple in triangles:
            last = max(turn_index[d] for d in diag_tuple)
            (p1 := p1 + 1) if last % 2 == 0 else (p2 := p2 + 1)
        if p1 > p2:
            p1_wins += 1
        elif p2 > p1:
            p2_wins += 1
        else:
            ties += 1

total_games = p1_wins + p2_wins + ties
summary = pd.DataFrame({
    "Outcome": ["Player 1 win", "Player 2 win", "Tie"],
    "Count": [p1_wins, p2_wins, ties],
    "Probability": [p1_wins/total_games, p2_wins/total_games, ties/total_games]
})
print(summary)

        Outcome  Count  Probability
0  Player 1 win   9632     0.608081
1  Player 2 win    384     0.024242
2           Tie   5824     0.367677
